In [ ]:
import sys
import os
import random
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
if os.path.exists('./core'):
    os.remove('./core')

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
    sys.path.append('../symbolicregression/')

# Install

In [ ]:
%%capture
if IN_COLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !git clone https://github.com/facebookresearch/symbolicregression symbolic
    %mv ./symbolic/* ./
    %rm -rf symbolic
    !conda env create --name symbolic regression --file=environment.yml
    !conda init
    !activate symbolic
    !pip install git+https://github.com/pakamienny/sympytorch
    !conda install -c conda-forge pysr
    !conda install -c conda-forge julia
    !python3 -m pysr install

In [ ]:
#import pysr
#from pysr import PySRRegressor
#pysr.julia_helpers.init_julia()

In [ ]:
from utils import *

# Costmap Experment

In [ ]:
#code by Muhammad 

In [ ]:
set_seed(42)
transformer=get_transformer_model(max_input_points=10000,n_trees_to_refine=3000)

In [ ]:
num_cells=256
cost_map  = create_cost_map(num_cells=num_cells)
plotential_matrix= np.zeros((num_cells,num_cells))
for i in range(num_cells):
    for j in range(num_cells):
        plotential_matrix[i,j] = get_potential(cost_map[i,j])

X = np.linspace(0, 5.12, num_cells)
Y = np.linspace(0, 5.12, num_cells)
X, Y = np.meshgrid(X, Y)


# # show image potential of cost map
plotting_plotential_matrix = np.zeros((num_cells,num_cells))
for i in range(num_cells):
    for j in range(num_cells):
        plotting_plotential_matrix[num_cells-1-i,j] = plotential_matrix[i,j]


In [ ]:
Z  = plotting_plotential_matrix[:]
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
#ax.contour3D(X, Y, Z_approx, 50,cmap=cm.coolwarm)
ax.contour3D(X, Y, Z, 50,cmap=cm.coolwarm)
plt.show()

In [ ]:
Z  = plotting_plotential_matrix[:]
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
#ax.contour3D(X, Y, Z_approx, 50,cmap=cm.coolwarm)
ax.contour3D(X, Y, Z, 50,cmap=cm.coolwarm)
ax.view_init(azim=0, elev=90)
plt.axis('off')
plt.show()

In [ ]:
cm_df = np.stack([plotential_matrix.flatten(),X.flatten(),Y.flatten()]).T
cm_df = pd.DataFrame(cm_df)
cm_df.columns='j2,x,y'.split(',')
cm_df = cm_df.sort_values('j2')[::5]
cm_df = cm_df.reset_index().drop('index',1)
n_train = int(len(cm_df)*0.9)
train_index = np.random.choice(cm_df.index,n_train)
test_index = np.array([i for i in cm_df.index if i not in train_index])
cm_df_train = cm_df.iloc[train_index]
cm_df_test = cm_df.iloc[test_index]

In [ ]:
np.squeeze(cm_df_train[['j2']].values).shape

In [ ]:
t = perf_counter()
transformer.fit(cm_df_train[['x','y']].values,np.squeeze(cm_df_train[['j2']].values))
t = perf_counter() - t

In [ ]:
cm_df_train[['x','y']].values.shape

In [ ]:
replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/",'x_0':'x','x_1':'y'}
transformer_str = transformer.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
for op,replace_op in replace_ops.items():
    transformer_str = transformer_str.replace(op,replace_op)

In [ ]:
cm_model_transformer = round_expr(sp.parse_expr(transformer_str),5)

In [ ]:
str(cm_model_transformer)

In [ ]:
j2_approx_tranfromer=sp.lambdify(['x','y'],cm_model_transformer,"numpy")

In [ ]:
try:
    approx_train = j2_approx_tranfromer(cm_df_train['x'].values,cm_df_train['y'].values)
    train_err = mse(approx_train,cm_df_train['j2'])
    approx_test = j2_approx_tranfromer(cm_df_test['x'].values,cm_df_test['y'].values)
    test_err = mse(approx_test,cm_df_test['j2'])
    print(train_err,test_err)
except Exception as e:
    print(e)

In [ ]:
fig = plt.figure(figsize=(10,10))
arr = cm_df_train.values
ax = fig.add_subplot(111, projection='3d')
s=35
ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_tranfromer(arr[:,1],arr[:,2]),color='red',label='transformer',s=s)
ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',label='j2',s=s,alpha=.5)

arr = cm_df_test.values

ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_tranfromer(arr[:,1],arr[:,2]),color='red',label='transformer (test)',marker='x',s=s)
ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',label='j2',marker='x',s=s,alpha=.5)

# Set the azimuth and elevation angles
#ax.view_init(azim=90, elev=30)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
X  = np.linspace(0, 5.12, num_cells)
Y  = np.linspace(0, 5.12, num_cells)
Z_approx = np.zeros((num_cells,num_cells))
for i in range(num_cells):
    for j in range(num_cells):
        Z_approx[num_cells-1-i,j] = j2_approx_tranfromer(X[i],Y[j])

In [ ]:
 j2_approx_tranfromer(X[i],Y[j])

In [ ]:

Z  = plotting_plotential_matrix[:]
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
ax.contour3D(X, Y, Z_approx, 50,cmap=cm.coolwarm,label='approximation of j2')
ax.contour3D(X, Y, Z, 50,cmap='binary',label='j2')
plt.legend(loc='best')

In [ ]:

Z  = plotting_plotential_matrix[:]
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
ax.contour3D(X, Y, Z_approx, 50,cmap=cm.coolwarm)
ax.contour3D(X, Y, Z, 50,cmap='binary')
ax.view_init(azim=0, elev=90)
plt.axis('off')
plt.show()